# Librerias

In [ ]:
!pip install 'dtreeviz'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

#Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import dtreeviz.trees as dtreeviz
import scipy.stats as ss


#modelos y métricas
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score, precision_recall_curve, roc_curve
from sklearn.metrics import confusion_matrix, classification_report
#preprocesamiento
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split


#configuración warnings
import warnings
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Importación de datasets

In [ ]:
url_train = 'https://raw.githubusercontent.com/FrancoSecchi/7506R-1C2023-GRUPO02/checkpoint-2/checkpoint-2/hotels-train.csv'
url_test = 'https://raw.githubusercontent.com/FrancoSecchi/7506R-1C2023-GRUPO02/checkpoint-2/checkpoint-2/hotels-test.csv'
df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)

In [ ]:
df_hotels_train = df_train.copy()
df_hotels_test = df_test.copy()

In [ ]:
df_hotels_train.columns.to_list()

['Unnamed: 0',
 'hotel',
 'lead_time',
 'arrival_date_year',
 'arrival_date_month',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'meal',
 'market_segment',
 'distribution_channel',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'reserved_room_type',
 'assigned_room_type',
 'booking_changes',
 'deposit_type',
 'days_in_waiting_list',
 'customer_type',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'reservation_status_date',
 'id',
 'is_canceled']

In [ ]:
df_hotels_train.drop(columns=['Unnamed: 0'], inplace=True)
df_hotels_test.drop(columns=['Unnamed: 0'], inplace=True)

# Funciones

In [ ]:
def cramers_v(categorical_var1: pd.Series, categorical_var2: pd.Series) -> float:
    """
    Calcula el coeficiente de Cramér's V entre dos variables categóricas.

    Parámetros:
    - categorical_var1: Serie de Pandas que representa la primera variable categórica.
    - categorical_var2: Serie de Pandas que representa la segunda variable categórica.

    Devuelve:
    - Coeficiente de Cramér's V (float) que mide la asociación entre las dos variables categóricas.
    """

    confusion_matrix = pd.crosstab(categorical_var1,categorical_var2)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

In [ ]:
# Funcion que reemplaza la variable "reservation_status_date" por una version numerica de si misma y la dropea
def replace_reservation_date(df):
  df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

  df['reservation_month'] = df['reservation_status_date'].dt.month
  df['reservation_year'] = df['reservation_status_date'].dt.year
  df['reservation_day'] = df['reservation_status_date'].dt.day

  df.drop(['reservation_status_date'], axis=1, inplace=True)

# Feature Engineering - Transformación de Datos

In [ ]:
#Droppeamos la variable ID, no nos ayuda a la transformacion de datos
df_hotels_train.drop(['id'], axis=1, inplace=True)

### **Columnas Droppeadas - Analisis de Correlación y Relevancia**

- Para ayudar a nuestra decision sobre que columnas droppear, observamos su nivel de correlacion con la variable target (is_canceled), ademas de esto tomaremos en cuenta si creemos que pueda aportar informacion valiosa para determinar si el cliente cancela la orden.



In [ ]:
# Correlacion de variables numericas
df_hotels_train.corr()['is_canceled'].abs().sort_values(ascending = False)

is_canceled                       1.000000
lead_time                         0.285630
total_of_special_requests         0.235391
required_car_parking_spaces       0.222230
booking_changes                   0.154349
previous_cancellations            0.137619
is_repeated_guest                 0.087756
previous_bookings_not_canceled    0.065778
adults                            0.059382
days_in_waiting_list              0.057142
adr                               0.048961
babies                            0.031658
stays_in_week_nights              0.030168
arrival_date_year                 0.017872
stays_in_weekend_nights           0.007100
arrival_date_week_number          0.004993
arrival_date_day_of_month         0.004906
children                          0.004268
Name: is_canceled, dtype: float64

In [ ]:
var_cualitative = [columna for columna in df_hotels_train.columns if not np.issubdtype(df_hotels_train[columna].dtype, np.number)]
var_cualitative.append('is_canceled')

In [ ]:
df_hotels_train_copy = df_hotels_train.copy()
df_hotels_train_copy.loc[df_hotels_train_copy.loc[:,"is_canceled"]==1,"is_canceled"] = "Si"
df_hotels_train_copy.loc[df_hotels_train_copy.loc[:,"is_canceled"]==0,"is_canceled"] = "No"

result = {}
corr_matrix = np.zeros((len(var_cualitative), len(var_cualitative)))

for i, var1 in enumerate(var_cualitative):
    for j, var2 in enumerate(var_cualitative):
        corr_matrix[i, j] = cramers_v(df_hotels_train_copy[var1], df_hotels_train_copy[var2])
        result[var1] = corr_matrix[i, j]

cualitative_corr = pd.Series(result).abs().sort_values(ascending=False)

In [ ]:
# Correlacion variables cualitativas
cualitative_corr

is_canceled                0.999966
reservation_status_date    0.469851
deposit_type               0.432740
market_segment             0.266809
assigned_room_type         0.208305
distribution_channel       0.191431
hotel                      0.144122
customer_type              0.141083
arrival_date_month         0.073392
reserved_room_type         0.066721
meal                       0.039765
dtype: float64

#### **Variables Cuantitativas a Borrar**

- arrival_date_year: 
> El año en que la persona llega al hotel no deberia influir mucho, ya que no hay un año con niveles de cancelaciones mas alto que otros, este feature no aporta mas valor que por ejemplo el numero de semana o mes del año de cuando se hospeda el cliente.

- days_in_waiting_list:
> Ademas de tener un nivel de correlacion bajo, el numero de dias en la lista de espera cuenta la diferencia de dias entre el momento en que la reserva fue creada en la DB del hotel hasta que fuese CONFIRMADA.



#### **Variables Categóricas a Borrar**
- assigned_room_type:
> Representa solamente el codigo de la habitacion asignada al usuario, esta variable suele ser igual a la habitacion reservada, sin embargo si difiere es mas que todo por peticion del usuario y/o niveles de disponibilidad del hotel informacion que esta mejor represantada en el campo `booking_changes`.
- meal:
> No consideramos que tenga una mayor correlacion con una decision sobre cancelar o no una reserva, simplemente representa el tipo de comida que reserva el cliente (Debatir).

In [ ]:
#Columnas no utiles
bad_cols = ['arrival_date_year','days_in_waiting_list','assigned_room_type', 'meal']
df_hotels_train.drop(bad_cols, axis=1, inplace=True)
df_hotels_test.drop(bad_cols, axis=1, inplace=True)

In [ ]:
df_hotels_train.columns.to_list()

['hotel',
 'lead_time',
 'arrival_date_month',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'market_segment',
 'distribution_channel',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'reserved_room_type',
 'booking_changes',
 'deposit_type',
 'customer_type',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'reservation_status_date',
 'is_canceled']

## Encoding y Normalizacion - Data Preparation

> Para esta etapa, separaremos nuestro dataset en 2 partes, las variables categoricas y las variables cuantitativas, esto puramente para un manejo y manipulacion de los datos mas comodos y acto seguido procederemos a mergearlo nuevamente en un nuevo dataset ya preparado para ser consumido por modelos.

### **DATASET CATEGORICO**

> Para este dataset, estaremos realizando `one-hot-encoding` para que las variables NO numericas sean aptas para ser interpretadas por el modelo.

In [ ]:
cols_categoricas = [columna for columna in df_hotels_train.columns if not np.issubdtype(df_hotels_train[columna].dtype, np.number)]
df_categorico = df_hotels_train[cols_categoricas]
df_categorico.head()

,hotel,arrival_date_month,market_segment,distribution_channel,reserved_room_type,deposit_type,customer_type,reservation_status_date
0,City Hotel,Sep,Online TA,TA/TO,D,No Deposit,Transient,2016-08-25
1,Resort Hotel,Nov,Groups,Direct,A,No Deposit,Transient,2015-11-01
2,City Hotel,Oct,Online TA,TA/TO,F,No Deposit,Transient,2016-10-14
3,City Hotel,Dec,Groups,TA/TO,A,Non Refund,Transient,2016-11-29
4,City Hotel,Mar,Offline TA/TO,TA/TO,A,No Deposit,Transient-Party,2017-03-26


> Podemos observar que la variable `reservation_status_date` se encuentra en tipo `datetime`, para hacerlo mas amigable al modelo, procedemos a transformarlo en columnas numericas.


**NOTA: Esta variable previamente habiamos establecido que no la ibamos a tomar en cuenta, sin embargo observamos que tiene un alto nivel de correlacion con la variable target, por lo que hemos decidido tomarla en cuenta para el pre-procesamiento del dataset.**

####Transformaciones de fechas

> Acomodamos las representaciones de las variables `reservation_status_date` y `arrival_date_month`

In [ ]:
#Modificamos reservation_status_date en test y train
replace_reservation_date(df_categorico)
replace_reservation_date(df_hotels_test)

In [ ]:
df_categorico.head()

,hotel,arrival_date_month,market_segment,distribution_channel,reserved_room_type,deposit_type,customer_type,reservation_month,reservation_year,reservation_day
0,City Hotel,Sep,Online TA,TA/TO,D,No Deposit,Transient,8,2016,25
1,Resort Hotel,Nov,Groups,Direct,A,No Deposit,Transient,11,2015,1
2,City Hotel,Oct,Online TA,TA/TO,F,No Deposit,Transient,10,2016,14
3,City Hotel,Dec,Groups,TA/TO,A,Non Refund,Transient,11,2016,29
4,City Hotel,Mar,Offline TA/TO,TA/TO,A,No Deposit,Transient-Party,3,2017,26


> Observamos que arrival_date_month se encuentra descrita por los nombres de los meses, esto al momento de hacer dummies, resultaria en un numero grande de variables, por ende pasaremos a hacerla a ser representada por los numeros de los meses.

In [ ]:
month_to_num = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
# Mapeamos los numeros de los meses al dataset de train y el de test
df_categorico['arrival_date_month'] = df_categorico['arrival_date_month'].map(month_to_num)
df_hotels_test['arrival_date_month'] = df_hotels_test['arrival_date_month'].map(month_to_num)

In [ ]:
df_categorico.head()

,hotel,arrival_date_month,market_segment,distribution_channel,reserved_room_type,deposit_type,customer_type,reservation_month,reservation_year,reservation_day
0,City Hotel,9,Online TA,TA/TO,D,No Deposit,Transient,8,2016,25
1,Resort Hotel,11,Groups,Direct,A,No Deposit,Transient,11,2015,1
2,City Hotel,10,Online TA,TA/TO,F,No Deposit,Transient,10,2016,14
3,City Hotel,12,Groups,TA/TO,A,Non Refund,Transient,11,2016,29
4,City Hotel,3,Offline TA/TO,TA/TO,A,No Deposit,Transient-Party,3,2017,26


#### Encoding de Variables
> Ejecucion del `one-hot-encoding` a traves de dummies para cada variable

In [ ]:
# Evaluar si el reservation_year es necesario hacerle dummies.
dummy_columns = ["hotel","market_segment","distribution_channel", "reserved_room_type", "deposit_type", "customer_type"]
#Train
df_categorico = pd.get_dummies(df_categorico, columns=dummy_columns, drop_first=True)
#Test
df_hotels_test = pd.get_dummies(df_hotels_test, columns=dummy_columns, drop_first=True)
df_categorico.columns

Index(['arrival_date_month', 'reservation_month', 'reservation_year',
       'reservation_day', 'hotel_Resort Hotel', 'market_segment_Complementary',
       'market_segment_Corporate', 'market_segment_Direct',
       'market_segment_Groups', 'market_segment_Offline TA/TO',
       'market_segment_Online TA', 'market_segment_Undefined',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'distribution_channel_TA/TO', 'distribution_channel_Undefined',
       'reserved_room_type_B', 'reserved_room_type_C', 'reserved_room_type_D',
       'reserved_room_type_E', 'reserved_room_type_F', 'reserved_room_type_G',
       'reserved_room_type_H', 'reserved_room_type_L',
       'deposit_type_Non Refund', 'deposit_type_Refundable',
       'customer_type_Group', 'customer_type_Transient',
       'customer_type_Transient-Party'],
      dtype='object')

### **DATASET CUANTITATIVO**

> Para este dataset, estaremos realizando `min-max scaling` para representar las distintas cardinalidades de las variables de una manera mas homogenea y facil de discernir para el modelo, evitando asi tener puntos de corte para condiciones que sean de cantidades muy distintas.

In [ ]:
cols_numericas = [columna for columna in df_hotels_train.columns if np.issubdtype(df_hotels_train[columna].dtype, np.number)]
# Creamos el df_numerico
df_numerico = df_hotels_train[cols_numericas]
df_numerico.drop('is_canceled', axis=1, inplace=True)
df_numerico.head()

,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,adr,required_car_parking_spaces,total_of_special_requests
0,49,37,5,1,2,1,0,0,0,0,0,0,115.5,0,1
1,4,44,31,0,1,2,0,0,0,0,0,0,42.0,1,0
2,25,42,13,0,1,2,2,0,0,0,0,0,229.0,0,2
3,26,51,12,2,5,2,0,0,0,0,0,0,75.0,0,0
4,104,12,19,2,5,2,0,0,0,0,0,0,75.0,0,1


In [ ]:
# Creamos el scaler
scaler = MinMaxScaler() 

# Sacamos del scaling a la otra variable booleana (no necesita scaling)
cols_numericas = df_numerico.columns.tolist()
cols_numericas.remove('is_repeated_guest')

# Alimentamos el scaler con los datos de TRAIN.
scaler.fit(pd.DataFrame(df_numerico[cols_numericas]))

# Aplicamos las transformaciones a el dataset de Train y al de TEST para nuestra prediccion de Kaggle.
df_numerico[cols_numericas]=scaler.transform(df_numerico[cols_numericas])
df_hotels_test[cols_numericas]=scaler.transform(df_hotels_test[cols_numericas])

###**MERGEO DATASET ENTRENAMIENTO**

> Volvemos a unir en un dataset que ya esta preparado para ser splitteado y para entrenar modelos y buscar hiperparametros.


In [ ]:
x_model = pd.concat([df_categorico, df_numerico], axis=1)
y_model = df_hotels_train['is_canceled']

#Podemos ver que el numero de filas coincide.
print(f"Shape del X: {x_model.shape} y Shape del Y: {y_model.shape} -- [Coinciden: {x_model.shape[0] == y_model.shape[0]}]")

Shape del X: (59481, 44) y Shape del Y: (59481,) -- [Coinciden: True]


In [ ]:
x_model.head()

,arrival_date_month,reservation_month,reservation_year,reservation_day,hotel_Resort Hotel,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,...,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,adr,required_car_parking_spaces,total_of_special_requests
0,9,8,2016,25,0,0,0,0,0,0,...,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.490259,0.000,0.2
1,11,11,2015,1,1,0,0,0,1,0,...,0.018519,0.0,0.0,0,0.0,0.0,0.0,0.176919,0.125,0.0
2,10,10,2016,14,0,0,0,0,0,0,...,0.018519,1.0,0.0,0,0.0,0.0,0.0,0.974123,0.000,0.4
3,12,11,2016,29,0,0,0,0,1,0,...,0.018519,0.0,0.0,0,0.0,0.0,0.0,0.317602,0.000,0.0
4,3,3,2017,26,0,0,0,0,0,1,...,0.018519,0.0,0.0,0,0.0,0.0,0.0,0.317602,0.000,0.2


### **DATA SPLIT - TRAIN Y VALIDACION**
> Con el nuevo x_model & y_model podemos ahora empezar con el split en datos de entrenamiento y validacion.

Cosas a realizar: 

PRIORIDAD MAXIMA: REVISAR TODO LO QUE DROPPEAMOS Y PERSISTIRLO EN EL DE TEST, NO LAS IMPUTACIONES, SOLO LOS DROPS DE COLUMNAS INCLUYENDO TRANSFORMACIONES DE VARIABLES Y FECHAS ETC ETC.


1. Apoyandonos sobre las correlaciones y el paper, decidir que variables no nos sirven.   [DONE PERO REVISAR CON FRAN]
2. Encoding de las variables categoricas y normalización de las cuantitativas [DONE PERO REVISAR CON FRAN]
2.5 SPLIT DE DATA
3. Construccion del arbol y optimización de los hiperparametros de k-folds. 
4. Seleccionar el mejor arbol y testear.

# Clasificación - Entrenamiento

# Árbol de decisión